In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df =pd.read_csv('Ecommerce Dataset.csv')
df_new=pd.read_csv('TBC_Ecommerce.csv')

In [ ]:
df_new.head()

In [ ]:

df =df.rename(columns={'Order number': 'Order_number'}, inplace = False)

In [ ]:
df.shape

In [ ]:
df

In [ ]:
df['SKU'].value_counts()

In [ ]:
df1 =df.groupby(['Order_number','SKU'])[['SKU']].count().T
df1

In [ ]:
df1 =df.groupby(['Order_number','SKU']).count()[['Date placed']]
df1

In [ ]:
df2 =df.groupby(['Order_number','SKU']).count()
df2

In [ ]:
import matplotlib.pyplot as plt
keys =[SKU for SKU, df in df.groupby(['SKU'])]
plt.bar(keys,df.groupby(['SKU']).count()['Order number'])

In [ ]:
import matplotlib.pyplot as plt
keys =[SKU for SKU, df in df.groupby(['SKU'])]
pd.DataFrame(keys,df.groupby(['SKU']).count()['Order_number']).T

In [ ]:
SKU_group.count()['Order_number']

In [ ]:
'''
load apriori and association package from mlxtend. 
Used different dataset because mlxtend need data in below format. 

             itemname  apple banana grapes
transaction  1            0    1     1
             2            1    0     1  
             3            1    0     0
             4            0    1     0
             
 we could have used above data as well but need to perform operation to bring in this format instead of that used seperate data only.            
'''


from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df1 = pd.read_csv('TBC_Ecommerce.csv', encoding="ISO-8859-1")
df1.head()

In [ ]:
# data has many country choose any one for check..
df1.SKU.value_counts().head(15)

In [ ]:
#convert data in format which it require converting using pivot table and Quantity sum as values. fill 0 if any nan values

basket = pd.pivot_table(data=df1,index='Order number',columns='Product',values='Quantity', \
                        aggfunc='sum',fill_value=0)

In [ ]:
basket.head()

In [ ]:
#this to check correctness after binning it to 1 at below code..
basket['What the Dog Saw (Penguin)'].head()

In [ ]:
# we dont need quantity sum we need either has taken or not so if user has taken that item mark as 1 else he has not taken 0.

def convert_into_binary(x):
    if x > 0:
        return 1
    else:
        return 0

In [ ]:
basket_sets = basket.applymap(convert_into_binary)

In [ ]:
# above steps we can same item has quantity now converted to 1 or 0.
basket_sets['The Wolf of Wall Street'].head()

In [ ]:
#call apriori function and pass minimum support here we are passing 7%. means 7 times in total number of transaction that item was present.
frequent_itemsets = apriori(basket_sets, min_support=0.002, use_colnames=True)

In [ ]:
#it will generate frequent itemsets using two step approch
frequent_itemsets

In [ ]:
# we have association rules which need to put on frequent itemset. here we are setting based on lift and has minimum lift as 1
rules_mlxtend = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules_mlxtend.head()

In [ ]:
# rules_mlxtend.rename(columns={'antecedents':'lhs','consequents':'rhs'})

# as based business use case we can sort based on confidance and lift.
rules_mlxtend[ (rules_mlxtend['lift'] >= 4) & (rules_mlxtend['confidence'] >= 0.8) ]

Directed graph below is built for this rule and shown below. it will have always incoming and outcoming edges. Incoming edge(s) will represent antecedants and the stub (arrow) will be next to node.

In [ ]:
#plotting output in a graph plot.

import networkx as nx
import matplotlib.pyplot as plt  

def draw_graph(rules, rules_to_show):
    G1 = nx.DiGraph()
    color_map=[]
    N = 50
    colors = np.random.rand(N)    
    strs=['R0', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11']

    for i in range(rules_to_show):
        G1.add_nodes_from(["R"+str(i)])
        for a in rules.iloc[i]['antecedents']:
            G1.add_nodes_from([a])
            G1.add_edge(a, "R"+str(i), color=colors[i] , weight = 2)
        for c in rules.iloc[i]['consequents']:
            G1.add_nodes_from([c])
            G1.add_edge("R"+str(i), c, color=colors[i],  weight=2)

    for node in G1:
        found_a_string = False
        for item in strs: 
            if node==item:
                found_a_string = True
        if found_a_string:
            color_map.append('yellow')
        else:
            color_map.append('green')       

    edges = G1.edges()
    colors = [G1[u][v]['color'] for u,v in edges]
    weights = [G1[u][v]['weight'] for u,v in edges]

    pos = nx.spring_layout(G1, k=16, scale=1)
    nx.draw(G1, pos, edges=edges, node_color = color_map, edge_color=colors, width=weights, font_size=16, 
            with_labels=False)            

    for p in pos:  # raise text positions
        pos[p][1] += 0.07
        nx.draw_networkx_labels(G1, pos)
        plt.show()

draw_graph (rules_mlxtend, 10) 